# Generate Word Vectors For Disease Associate Gene Sentences

This notebook is designed to generate word vectors for disease associates gene (DaG) sentences. Using facebooks's fasttext, we trained word vectors using all sentences that contain a disease and gene mention. The model was trained using the following specifications:

| Parameter | Value |
| --- | --- |
| Size | 300 |
| alpha | 0.005 | 
| window | 2 |
| epochs | 50 |
| seed | 100 | 

# Set up the Environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import os
import pickle
import sys

sys.path.append(os.path.abspath('../../../modules'))

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

from gensim.models import FastText
from gensim.models import KeyedVectors

from utils.notebook_utils.dataframe_helper import load_candidate_dataframes, generate_embedded_df

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.learning.pytorch.rnn.rnn_base import mark_sentence
from snorkel.learning.pytorch.rnn.utils import candidate_to_tokens
from snorkel.models import Candidate, candidate_subclass

In [4]:
DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])

# Disease Associates Disease

This section loads the dataframe that contains all disease associates gene candidate sentences and their respective dataset assignments.

In [5]:
cutoff = 300
total_candidates_df = (
    pd.read_table("../dataset_statistics/data/all_dag_map.tsv.xz")
    .query("sen_length < @cutoff")
)
total_candidates_df.head(2)

,entrez_gene_id,gene_symbol,doid_id,doid_name,sources,hetionet,n_sentences,has_sentence,partition_rank,split,compound_mention_count,disease_mention_count,gene_mention_count,sentence_id,text,sen_length,candidate_id
0,1,A1BG,DOID:2531,hematologic cancer,NaN,0,8,1,0.8586,1,0.0,1.0,3.0,2129459,The number of H antigen sites per single red c...,172,176680
1,1,A1BG,DOID:2531,hematologic cancer,NaN,0,8,1,0.8586,1,0.0,1.0,3.0,2129459,The number of H antigen sites per single red c...,172,176675


# Train Word Vectors

This section trains the word vectors using the specifications described above.

In [9]:
words_to_embed = []
candidates = (
    session
    .query(DiseaseGene)
    .filter(
        DiseaseGene.id.in_(
            total_candidates_df
            .candidate_id
            .astype(int)
            .tolist()
        )
    )
    .all()
)

In [10]:
for cand in tqdm_notebook(candidates):
    args = [
                (cand[0].get_word_start(), cand[0].get_word_end(), 1),
                (cand[1].get_word_start(), cand[1].get_word_end(), 2)
    ]
    words_to_embed.append(mark_sentence(candidate_to_tokens(cand), args))

HBox(children=(IntProgress(value=0, max=3429890), HTML(value='')))

In [11]:
model = FastText(
    words_to_embed, 
    window=2, 
    negative=10, 
    iter=50, 
    sg=1, 
    workers=4, 
    alpha=0.005, 
    size=300, 
    seed=100
)

In [12]:
(
    model
    .wv
    .save_word2vec_format(
        "results/disease_associates_gene_word_vectors.bin", 
        fvocab="results/disease_associates_gene_word_vocab.txt", 
        binary=False
        )
)

In [13]:
model.wv.most_similar("diabetes")

/home/danich1/anaconda2/envs/snorkeling/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('diabetes.diabetes', 0.9513350129127502),
 ('diabetes.a', 0.8961078524589539),
 ('diabete', 0.8706765174865723),
 ('diabetes/obesity', 0.8492350578308105),
 ('diabetes.we', 0.8229795694351196),
 ('2-diabetes', 0.8153343200683594),
 ('obesity/diabetes', 0.7979199290275574),
 ('mellitus', 0.7894909977912903),
 ('diabetes.there', 0.7881076335906982),
 ('prediabetes', 0.7879585027694702)]

In [14]:
word_dict = {val[1]:val[0] for val in list(enumerate(model.wv.vocab.keys()))}
word_dict_df = (
    pd
    .DataFrame
    .from_dict(word_dict, orient="index")
    .reset_index()
    .rename({"index":"word", 0:"index"}, axis=1)
)
word_dict_df.to_csv("results/disease_associates_gene_word_dict.tsv", sep="\t", index=False)
word_dict_df.head(2)

,word,index
0,overexpression,0
1,of,1
